#### Note: read model-prompt-parser.ipynb first before reading this notebook.

In [58]:
import json
import textwrap

## Initialize LLM

In [4]:
import dotenv
dotenv.load_dotenv()
from langchain_openai import ChatOpenAI, AzureChatOpenAI

llm = AzureChatOpenAI(
    max_tokens=16000,
    temperature=0.5,
    deployment_name="gpt-4o-b",
    model_name="gpt-4o",
    # deployment_name="gpt-35-turbo",
    # model_name="gpt-3.5-turbo",
)
print(f'Using Azure endpoint: {llm.azure_endpoint}')

Using Azure endpoint: https://summarilyazureopenai.openai.azure.com


In [5]:
# Test if LLM responds OK
prompt = "What is your training data cutoff date?"
response = llm(prompt)
print(response.content)

My training data cutoff date is **October 2021**. This means I was trained on information available up until that time, and I don't have knowledge of events, developments, or data created or released after that date. Let me know if you have any questions!


## Importing Data

In [7]:
import pandas as pd
df = pd.read_csv("Data.csv")

In [8]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LangChain Chains

### 1. LMChain

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [22]:
template_string = "What is the best name to describe a company that makes {product}? \
    answer in one name which you think is the best"
prompt = ChatPromptTemplate.from_template(template_string)

In [23]:
chain = LLMChain(llm=llm, prompt=prompt)

In [24]:
product = "Waterproof Phone Pouch"
response = chain(product)
print(response["text"])

AquaShield


### 2. SimpleSiquentialChain

In [31]:
from langchain.chains import SimpleSequentialChain

In [27]:
llm = AzureChatOpenAI(max_tokens=16000, temperature=0.5, 
                      deployment_name="gpt-4o-b", model_name="gpt-4o")

In [29]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [30]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [33]:
overall_simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two], verbose=True)

In [37]:
response = overall_simple_chain("Waterproof Phone Pouch")
print(response["output"])



> Entering new SimpleSequentialChain chain...
Choosing the best name for a company that makes waterproof phone pouches depends on your brand identity, target audience, and the message you want to convey. Here are some ideas that combine creativity, clarity, and relevance:

### Functional & Descriptive Names:
1. **AquaGuard**
2. **HydroShield**
3. **DryPocket**
4. **WaterSafe**
5. **AquaVault**
6. **SealTech**
7. **HydroLock**
8. **DryDock Gear**
9. **PhoneFloat**
10. **AquaArmor**

### Modern & Trendy Names:
1. **SplashProof**
2. **WaveSafe**
3. **DriftCase**
4. **FloatSafe**
5. **AquaWrap**
6. **HydroCase**
7. **OceanSeal**
8. **BlueBarrier**
9. **LiquidLock**
10. **StreamShield**

### Adventure-Inspired Names:
1. **NomadDry**
2. **AquaTrail**
3. **WetProof Gear**
4. **TideSafe**
5. **AdventureSeal**
6. **HydroVenture**
7. **OceanProof**
8. **GearFloat**
9. **Voyager Pouch**
10. **ExplorerDry**

### Minimalist & Elegant Names:
1. **AquaPod**
2. **DryZone**
3. **Hydra**
4. **PureSeal

### 3. SequentialChain

In [44]:
from langchain.chains import SequentialChain

In [52]:
first_prompt = ChatPromptTemplate.from_template("Translate the followin review to \
                                                English: \n\n{Review}")

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [53]:
second_prompt = ChatPromptTemplate.from_template("Summerize the following review in one sentence \
                                                \n\n{English_Review}")

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [54]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [55]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [56]:
overall_chain = SequentialChain(chains=[chain_one, chain_two, chain_three, chain_four], 
                                input_variables=["Review"],
                                output_variables=["English_Review", "summary", "language", "followup_message"],
                                verbose=True)

In [72]:
sampple_review = df.Review[5]
response = overall_chain(sampple_review)



> Entering new SequentialChain chain...

> Finished chain.


In [73]:
for key, value in response.items():
    value = "\n".join(textwrap.wrap(value))
    print(f"{key}: \n {value} \n")

Review: 
 Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre.
J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !? 

English_Review: 
 I find the taste mediocre. The foam doesn't hold, which is strange. I
buy the same ones in stores, and the taste is much better...   Old
batch or counterfeit!? 

summary: 
 The reviewer finds the taste mediocre, the foam unstable, and suspects
the product might be an old batch or counterfeit compared to better-
quality store-bought versions. 

language: 
 Le commentaire est en **français**. 

followup_message: 
 Merci pour votre retour détaillé. Nous sommes désolés d'apprendre que
le produit n'a pas répondu à vos attentes. La qualité est une priorité
pour nous, et nous prenons vos remarques très au sérieux. Concernant
vos doutes sur la fraîcheur ou l'authenticité du produit, pourriez-
vous nous fournir plus de détails, comme le numéro de lot ou la date
d'expiration indiquée sur l'emballage ? Cela